In [1]:
import pandas as pd
import numpy as np
import pickle
import nltk
import re
from nltk.tokenize import RegexpTokenizer
from scipy.sparse import hstack
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
import warnings
warnings.filterwarnings('ignore')
import sklearn as sk
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import TruncatedSVD
import datetime
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif
from sklearn.model_selection import GridSearchCV
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split

In [2]:
DATA_PATH = "../data/"

In [3]:
train = pd.read_csv(DATA_PATH + 'preprocessed/train_ling.csv')
test = pd.read_csv(DATA_PATH + 'preprocessed/test_ling.csv')

In [4]:
targets = train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].values

In [5]:
X_train, X_dev, y_train, y_dev = train_test_split(train['comment_text'], targets, test_size=0.3, random_state=42)

In [6]:
tokenizer = RegexpTokenizer(r'[A-Z]{2,}(?![a-z])|[A-Z][a-z]+(?=[A-Z])|[\'\w\-]+')
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2),tokenizer=tokenizer.tokenize, min_df=1)
analyze = bigram_vectorizer.build_analyzer()
X_train = bigram_vectorizer.fit_transform(X_train)
print("Uni- and bigrams shape: ", X_train.shape)

Uni- and bigrams shape:  (111699, 1947077)


In [7]:
X_dev = bigram_vectorizer.transform(X_dev)

In [ ]:
model_to_set = OneVsRestClassifier(SVC())

parameters = {'estimator__C': [0.1, 1, 10, 100, 1000], 'estimator__gamma': np.linspace(0.1, 1.0, num=3), 'estimator__kernel': ['rbf', 'linear', 'sigmoid']}

model_tuning = GridSearchCV(model_to_set, param_grid=parameters,
                             scoring ='roc_auc', cv=5, verbose=3)

In [ ]:
start_time = datetime.datetime.now()
model_tuning.fit(X_train, y_train)
print('Time elapsed:', (datetime.datetime.now() - start_time).total_seconds())
print("Best parameters set found on development set:")
print(model_tuning.best_params_)
print("Best ROC AUC score:")
print(model_tuning.best_score_)
y_pred = model_tuning.best_estimator_.predict(X_dev)

Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] estimator__C=0.1, estimator__gamma=0.1, estimator__kernel=rbf ...
